In [2]:
import pandas as pd
Tenders = pd.read_excel("../../../data/UpdatedAgainTenders.xlsx")

In [13]:
Tenders.columns

Index(['Client Agency', 'Client Agency Address', 'Developing Agency Parent',
       'Reference Number', 'Type of Work', 'Contract Title', 'Description',
       'Tender Closing Date', 'UNSPSC Code', 'UNSPSC Title',
       'Procurement Method', 'Period Contract', 'CUA Contract',
       'Original Contract Value', 'Revised Contract Value', 'Region/s',
       'Awarded Date', 'Start Date', 'Initial Expiry Date', 'Expiry Date',
       'Number of Submissions', 'Panel Contract', 'Contract Developer',
       'Contract Manager', 'Supplier Name', 'Supplier ABN', 'Supplier Address',
       'DCSP Contract', 'supplier contract price', 'Number of Variations',
       'Variation Amount', 'Contract Expenditure', 'Tender Number',
       'Tender Open Date', 'Tender Close Date', 'Tender ID', 'TenderLink'],
      dtype='object')

In [97]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', '¯', '♦', '¤', '▲', '¸', '¾', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', '¹', '≤', '‡', '√', ]

def remove_punctuation(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, '')
    return x

stop_words = sw.words()

# Adapted from Joyce's 'deal_data.py' file
import re
def preprocess(content):
    text = re.findall(r'>([^<]+)<', content)
    text = [t.strip() for t in text if t.strip()]
    text = " ".join(text)
    text = re.sub("\xa0", '', text)
    text = remove_punctuation(text)
    text = text.lower()
    tokens = word_tokenize(text)
    filtered_sentence = [w for w in tokens if not w in stop_words]
    return filtered_sentence

[nltk_data] Downloading package punkt to
[nltk_data]     /home/ucc/maxichat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ucc/maxichat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [104]:
sentences = [preprocess(content) for content in Tenders["Description"]]

In [109]:
len(sentences) == len(Tenders)

True

In [112]:
from gensim.models import FastText
#skip gram
ft_model = FastText(sentences, vector_size=100, window=5, min_count=5, workers=2, sg=1)

In [120]:
ft_model.wv['austrlia']

array([-0.01129239, -0.213312  ,  0.76592624,  0.18122433,  0.10902726,
       -0.20638226, -0.15469751, -0.04000745,  0.43963322, -0.26372322,
        0.09150757,  0.09030248, -0.10152188,  0.17080572, -0.30549133,
        0.14313504,  0.23966828, -0.42562246, -0.33569124, -0.7142659 ,
       -0.08556675,  0.4568162 , -0.00122319, -0.01923689, -0.01153962,
       -0.22891451, -0.81238645, -0.30243582,  0.10705744,  0.23443142,
       -0.08050987, -0.19760934,  0.32736307,  0.2163056 , -0.23116644,
        0.40134633, -0.1602959 , -0.0533842 , -0.2949559 ,  0.0500124 ,
        0.1069188 ,  0.04872245,  0.16048524, -0.36198002, -0.10417377,
        0.01472887, -0.18377806, -0.30446166,  0.15558945, -0.1860763 ,
        0.63949454, -0.00593806,  0.3524825 , -0.04385049,  0.01688425,
       -0.2908308 , -0.14256573,  0.36038497, -0.35243732,  0.21467263,
        0.4709558 , -0.14009348,  0.06907006, -0.22690342, -0.06932667,
        0.15087384,  0.12290405, -0.28577664,  0.24426028,  0.90

In [137]:
# get a single embedding for each document by mean.
import numpy as np
embeddings = []
for sentence in sentences:
    s = np.zeros(100)
    i = 0
    for word in sentence:
        s += ft_model.wv[word]
        i += 1
    if i != 0:
        s = s/i
    embeddings.append(s)

In [ ]:
np.savetext("fasttext_emb.csv", embeddings, )